In [1]:
import os, sys, time, pickle
from pathlib import Path
from collections import Counter

PROJECT_DIR = Path(sys.path[0])/".."
DATA_DIR = PROJECT_DIR/"data"
SRC_DIR = PROJECT_DIR/"src"
sys.path.append(SRC_DIR)

%pylab inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
from googleapiclient.discovery import build
from typing import List, Dict

def get_playlist_items(api_key: str, playlist_id: str) -> List[Dict[str, str]]:
    """
    Fetch all videos from a YouTube playlist.
    
    Args:
        api_key: Your YouTube Data API key
        playlist_id: The ID of the playlist (found in the URL after list=)
        
    Returns:
        List of dictionaries containing video information
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None
    
    while True:
        # Get playlist items
        request = youtube.playlistItems().list(
            part='contentDetails,snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        
        # Extract video information
        for item in response['items']:
            video = {
                'title': item['snippet']['title'],
                'video_id': item['contentDetails']['videoId'],
                'url': f"https://www.youtube.com/watch?v={item['contentDetails']['videoId']}"
            }
            videos.append(video)
        
        # Check if there are more pages
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    
    return videos

# Example usage
if __name__ == "__main__":
    API_KEY = 'YOUR_API_KEY_HERE'
    PLAYLIST_ID = 'YOUR_PLAYLIST_ID_HERE'  # The part after 'list=' in the playlist URL
    
    try:
        videos = get_playlist_items(API_KEY, PLAYLIST_ID)
        for video in videos:
            print(f"Title: {video['title']}")
            print(f"URL: {video['url']}")
            print("---")
    except Exception as e:
        print(f"An error occurred: {str(e)}")